In [ ]:
# container_lan - Put container on LAN and forward LAN to Router zone.
# modify container config to use bridge networking on LAN.
# put first container on first LAN using IP address +1 of router LAN.
# forward Primary LAN Zone to Router Zone Default Allow All

import csclient
import ipcalc

cp = csclient.CSClient('container_lan')

projects = cp.get('config/container/projects/')
project_id = [x["_id_"] for x in projects if 'phate999/sdk' in x["config"]][0]
project = cp.get(f'config/container/projects/{project_id}/')
name = project["name"]
config = project["config"]

auto_setup = "version: '2.4'\nservices:\n  jupyter:\n    image: 'phate999/sdk'\n    volumes:\n     - ${CONFIG_STORE}"

# Check for auto_setup compose:
if config == auto_setup:
    
    # Get IP info for project
    lan = cp.get('config/lan/0')
    lan_name = lan["name"]
    lan_ip = lan["ip_address"]
    netmask = lan["netmask"]
    lan_id = lan["_id_"]
    addr = ipcalc.IP(lan_ip, mask=netmask)
    subnet = str(addr.guess_network())
    
    # Ensure IP is not in client list
    clients = cp.get('status/lan/clients')
    clients = [x["ip_address"] for x in clients if
               not x["ip_address"].startswith('fe80')]  # No IPv6
    my_lan = addr + 1
    while str(my_lan) in clients:
        my_lan = my_lan + 1
    
    # Modify project config
    lan_config = f"version: '2.4'  \nservices:  \n  jupyter:  \n    image: 'phate999/sdk'  \n    networks:\n      lannet:\n        ipv4_address: {my_lan}\n    volumes:  \n     - ${{CONFIG_STORE}}  \n     - 'shared-data:/home/jovyan'  \nvolumes:  \n  shared-data:  \n    driver: local\nnetworks:\n  lannet:\n    driver: bridge\n    driver_opts:\n      com.cradlepoint.network.bridge.uuid: {lan_id}\n    ipam:\n      driver: default\n      config:\n        - subnet: {subnet}\n          gateway: {lan_ip}"
    resp = cp.put(f'config/container/projects/{project_id}/config', lan_config)
    cp.log(f'Auto-Setup Compose found.  Modified Project {name} to use IP Address {my_lan} on {lan_name}')
else:
    cp.log('Auto-Setup Compose not found.  No modifications made.')


